In [ ]:
input_root = "./input_recordings"                 # root folder containing recordings
output_root = "./processed_segments_csv"          # where CSV and wav outputs will be saved

channel_num=2 # how many channels is waited to be in the wav file

noise_profile_fn = "noise_profiles_25_01_14.zip" # noise profiles generated by compute_noise_profiles.ipynb
b_draw=False # if to draw spectorgrams for debug.


In [ ]:
import os
from pathlib import Path
from pydub import AudioSegment
import os
import numpy as np
import librosa
from scipy.signal import butter, filtfilt
import glob
import pandas as pd

# Supported audio file formats
supported_formats = [".wav"]

# Create the output folder if it doesn't exist
os.makedirs(output_root, exist_ok=True)



In [ ]:
df_noise=pd.read_csv(noise_profile_fn)
df_noise.head()


In [ ]:
def highpass_filter(y, sr, cutoff=250, order=4):
    #Applies a high-pass filter to the data.
    nyquist = 0.5 * sr
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return filtfilt(b, a, y)
        


In [ ]:
def analyze_spectrogram_segment(segment, sr, n_fft, hop_length, threshold=17): # 15 is more permissive, 17-18 filters a lot
    """
    Analyze a single spectrogram segment.
    """
    # Aggregate spectrum by taking the maximum of each frequency bin
    #aggregated_spectrum = np.max(segment, axis=1)
    aggregated_spectrum = np.mean(segment, axis=1)

    # Frequencies calculation
    freqs = np.fft.rfftfreq(n=n_fft, d=1.0/sr)

    # Selecting the 300-1500 Hz range
    target_indices = np.where((freqs >= 300) & (freqs <= 1500))[0]
    target_spectrum = aggregated_spectrum[target_indices]

    # Find the first local minimum
    if target_spectrum[0] >= target_spectrum[1]:
        local_min_indices = librosa.util.peak_pick(-target_spectrum, pre_max=3, post_max=3, pre_avg=5, post_avg=5, delta=0.3, wait=1)
        first_local_min_index = target_indices[local_min_indices[0]] if len(local_min_indices) > 0 else target_indices[0]
    else:
        first_local_min_index = target_indices[0]

    # Find the maximum value after the first local minimum
    #subsequent_spectrum = aggregated_spectrum[first_local_min_index:]
    subsequent_spectrum = aggregated_spectrum[first_local_min_index:target_indices[-1]]
    #max_index = first_local_min_index + np.argmax(subsequent_spectrum)
    if subsequent_spectrum.size > 0:  # Check that it is not empty
        max_index = first_local_min_index + np.argmax(subsequent_spectrum)
    else:
        print("Warning: subsequent_spectrum is empty, using first_local_min_index as max_index")
        max_index = first_local_min_index  # Default value

    # Find the local minimum after the maximum
    remaining_spectrum = aggregated_spectrum[max_index:]
    local_min_indices = librosa.util.peak_pick(-remaining_spectrum, pre_max=3, post_max=3, pre_avg=5, post_avg=5, delta=0.3, wait=1)
    local_min_index = max_index + local_min_indices[0] if len(local_min_indices) > 0 else max_index + np.argmin(remaining_spectrum)

    # Calculate decibel difference
    max_db = aggregated_spectrum[max_index]
    min_db = aggregated_spectrum[local_min_index]
    db_difference = max_db - min_db

    # PLOTTING ----------------------------
    
    # Visualization of aggregated spectrum and key points
    if b_draw:
        plt.figure(figsize=(10, 6))
        plt.plot(freqs, aggregated_spectrum, label='Aggregated Spectrum')
        plt.scatter(freqs[first_local_min_index], aggregated_spectrum[first_local_min_index], color='red', label=f'First Min: {min_db:.2f}')
        plt.scatter(freqs[max_index], aggregated_spectrum[max_index], color='green', label=f'Max: {max_db:.2f}, Max fr.: {freqs[max_index]:.2f}')
        plt.scatter(freqs[local_min_index], aggregated_spectrum[local_min_index], color='blue', label=f'Post-Max Min: {db_difference:.2f}')
        plt.legend()
        plt.title('Aggregated Spectrum with Local Min/Max Points')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude (dB)')
        plt.grid()
        plt.show()
    

    if freqs[max_index] > 1500:  # No fundamental frequency in the appropriate range
        return False

    #if (freqs[max_index] > 750) & (freqs[max_index] < 775):  # there is a continuous frequency (765 Hz) in every segment
    #    return False

    return db_difference > threshold




### spectrogram, etc.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
from librosa.display import specshow
from scipy.ndimage import uniform_filter1d

# Updated filter_audio_chunk function with spectrogram segmentation
def filter_audio_chunk(chunk, noise_profile):
    """
    Filter function that decides whether the given audio chunk contains a mosquito sound.
    """
    data = chunk.get_array_of_samples()
    dtype = data.typecode  # The type of the array, e.g., 'h' or 'i'
    y = np.array(data, dtype=np.float32)  # Always convert to float32

    # Normalization depending on the data type
    if dtype == 'h':  # 16-bit integer
        y = y / (2**15)  # Normalize between -1 and 1
    elif dtype == 'i':  # 32-bit integer
        y = y / (2**31)  # Normalize between -1 and 1
    else:
        raise ValueError(f"Unsupported data format: {dtype}")
        
    #y = np.array(chunk.get_array_of_samples(), dtype=np.float32) / (2**15)
    sr = chunk.frame_rate

    # 100 Hz high-pass filtering
    y = highpass_filter(y, sr)

    # Spectrogram calculation after highpass filter
    #n_fft = 512  # FFT window size for 8 kHz
    #xhop_length = 50  # Hop size
    n_fft = 1024  # FFT window size for 16 kHz
    hop_length = 50  # Hop size
    
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))

    if noise_profile is not None:
        # Check the dimension of the noise profile
        if noise_profile.shape[0] != S.shape[0]:
            raise ValueError("The number of rows in the noise profile and the spectrogram do not match.")

        # Subtracting the noise profile from each column (broadcasting)
        S_denoised = S / (noise_profile[:, np.newaxis] + 1e-80)  # Adding a small value to avoid division by zero
    else:
        #print("Warning: noise reduction was not done for the sound channel.")
        S_denoised = S
    
    S_db = librosa.amplitude_to_db(S_denoised, ref=np.max)

    #me_window_size=32
    #for i in range(S_db.shape[1]):
    #    moving_avg = uniform_filter1d(S_db[:, i], size=me_window_size, mode='reflect')
    #    S_db[:, i] = S_db[:, i] - moving_avg

    me_window_size = 2  # Moving window size
    nyquist = 0.5 * S_db.shape[0]  # Nyquist frequency
    cutoff = 1 / me_window_size  # Cutoff frequency based on window size
    b, a = butter(4, cutoff / nyquist, btype='low', analog=False)  # Low-pass filter
    # Filtering for each column vector
    for i in range(S_db.shape[1]):
        smoothed_signal = filtfilt(b, a, S_db[:, i])  # Filter forward-backward
        S_db[:, i] = S_db[:, i] - smoothed_signal  # Subtracting the filtered signal

    

    # PLOTTING ----------------------------
    
    # Display spectrogram
    if b_draw:
        plt.figure(figsize=(10, 6))
        librosa.display.specshow(S_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='linear', cmap='viridis')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram After Highpass Filter')
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        plt.show()
    
    # -------------------------------------

    mosquito_count = 0
    
    # Spectrogram slicing into 10 segments
    num_segments = 10
    segment_length = S_db.shape[1] // num_segments

    raw_segment_length = len(y) // num_segments

    for i in range(num_segments):
        start = i * segment_length
        end = start + segment_length if i < num_segments - 1 else S_db.shape[1]
        spectrogram_segment = S_db[:, start:end]

        raw_start = i * raw_segment_length
        raw_end = raw_start + raw_segment_length if i < num_segments - 1 else len(y)
        y_segment_max = np.max(np.abs(y[raw_start:raw_end]))

        # Analyze the spectrogram segment
        # discard the signal if it's too weak, don't analyze it
        #if (y_segment_max>0.02) and analyze_spectrogram_segment(spectrogram_segment, sr, n_fft, hop_length):
        if (y_segment_max>0.02) and analyze_spectrogram_segment(spectrogram_segment, sr, n_fft, hop_length):
            mosquito_count += 1

    return mosquito_count >= 3



In [ ]:
def split_audio_chunks(input_file, channel_num):
    """
    Read audio file, determine number of channels, split into 1-second chunks with 0.5-second overlap.
    """
    chunks = []
    metadata = []
    try:
        # Load audio file
        audio = AudioSegment.from_file(input_file)

        # Determine the number of channels
        num_channels = audio.channels
        print(f"File: {input_file}, Number of channels: {num_channels}")
        if num_channels!=channel_num:
            print("WARNING: incorrect channel number !!!")
            #return  [], [], num_channels

        # Separate all channels
        separated_channels = audio.split_to_mono()

        for channel_index, channel_audio in enumerate(separated_channels):
            #if channel_index<3:
            #    continue

            # Convert to 8kHz
            #channel_audio = channel_audio.set_frame_rate(8000)
            channel_audio = channel_audio.set_frame_rate(16000)

            duration_ms = len(channel_audio)

            # x-second window, y-second step
            window_size = 1000  # in milliseconds
            step_size = 500     # in milliseconds

            for start_ms in range(0, duration_ms - window_size + 1, step_size):
                end_ms = start_ms + window_size
                chunk = channel_audio[start_ms:end_ms]
                chunks.append(chunk)

                # Record metadata
                metadata.append({
                    "file": os.path.basename(input_file),
                    "channel": channel_index + 1,
                    "start_ms": start_ms,
                    "end_ms": end_ms
                })
            #break # channel
            
    except Exception as e:
        print(f"Error during splitting: {input_file} - {e}")

    return chunks, metadata, num_channels


In [ ]:
import torch
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps

model_silero_vad = load_silero_vad()

def filter_speech(chunk_16000):

    data = chunk_16000.get_array_of_samples()
    dtype = data.typecode  # The type of the array, e.g., 'h' or 'i'
    y = np.array(data, dtype=np.float32)  # Always convert to float32

    # Normalization depending on the data type
    if dtype == 'h':  # 16-bit integer
        y = y / (2**15)  # Normalize between -1 and 1
    elif dtype == 'i':  # 32-bit integer
        y = y / (2**31)  # Normalize between -1 and 1
    else:
        raise ValueError(f"Unsupported data format: {dtype}")
        
    #y = np.array(chunk_8000.get_array_of_samples(), dtype=np.float32) / (2**15)
    #sr = chunk_8000.frame_rate

    # 8 kHz -> 16 kHz conversion
    #chunk_16000 = librosa.resample(y, orig_sr=8000, target_sr=16000)
    #chunk_16000 = torch.tensor(chunk_16000, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    
    speech_timestamps = get_speech_timestamps(
      y,
      model_silero_vad,
      return_seconds=True,  # Return speech timestamps in seconds (default is samples)
    )
    
    if len(speech_timestamps):
        return True
    else:
        return False


In [ ]:
def anal_chunks(chunks, output_dir, base_filename, metadata, df_noise, b_save=False):
    """
    Save chunks to the specified folder with numbering.
    """
    os.makedirs(output_dir, exist_ok=True)
    not_selected_dir = os.path.join(os.path.dirname(output_dir), os.path.basename(output_dir) + "_not_selected")
    os.makedirs(not_selected_dir, exist_ok=True)

    speech_dir = os.path.join(os.path.dirname(output_dir), os.path.basename(output_dir) + "_speech")
    os.makedirs(speech_dir, exist_ok=True)

    sound_idxs=[]

    filtered_df = df_noise[(df_noise['filename'] == base_filename)]
    if filtered_df.empty:
        print(f"WARNING: No noise profile of file {base_filename}. That fact will highly reduce precision.")

    for idx, chunk in enumerate(chunks):
        md=metadata[idx]

        ch=md['channel']
        start=str(int(md['start_ms']))

        # Filter by filename and channel
        filtered_df = df_noise[(df_noise['filename'] == base_filename) & (df_noise['channel'] == ch)]
        if filtered_df.empty:
            #raise ValueError(f"No noise profile for channel {num_channels} of file {base_filename}.")
            #print(f"WARNING: No noise profile for channel {num_channels} of file {base_filename}. That fact will highly reduce precision.")
            noise_profile = None
        else:
            # Reconstruct profile based on frequency-bin
            noise_profile = filtered_df.sort_values(by="frequency_bin")['value'].values

        
        #if filter_speech(chunk):
        #    if b_save:
        #        chunk_speech_file = os.path.join(speech_dir, f"{base_filename}_{ch}_{start}.wav")            
                #chunk.export(chunk_speech_file, format="wav", parameters=["-ar", "16000", "-ac", "1", "-sample_fmt", "s16"])

        #    if idx % 50 == 0:
        #        pass
                #print(f"{idx}: Saved chunk (SPEECH): {chunk_speech_file}")
        #    continue
        
        if filter_audio_chunk(chunk, noise_profile):
            sound_idxs.append(idx)

            if b_save:
                chunk_output_file = os.path.join(output_dir, f"{base_filename}_{ch}_{start}.wav")
                chunk.export(chunk_output_file, format="wav", parameters=["-ar", "16000", "-ac", "1", "-sample_fmt", "s16"])

            if idx % 10 == 0:
                pass
                print(f"{idx}: Saved chunk (MOSQUITO SOUND): {chunk_output_file}")
        else:
            if b_save:
                not_selected_file = os.path.join(not_selected_dir, f"{base_filename}_{ch}_{start}.wav")
                #chunk.export(not_selected_file, format="wav", parameters=["-ar", "16000", "-ac", "1", "-sample_fmt", "s16"])

            if idx % 50 == 0:
                pass
                #print(f"{idx}: Not selected chunk: {not_selected_file}")

        #break
        
    return sound_idxs


In [ ]:

fns=glob.glob(os.path.join(input_root,"*.wav"))

for idx, input_fn in enumerate(fns):

    base_filename=os.path.basename(input_fn)
    output_file = os.path.join(output_root, f"{base_filename[:-4]}.csv")

    
    #If the output file already exists, continue with the next file
    if os.path.exists(output_file):
        print(f"already done: {input_fn}")
        continue

    # Splitting and filtering
    chunks, metadata, num_channels = split_audio_chunks(input_fn, channel_num=channel_num)
    if len(chunks)>0:

    
        sound_idxs=anal_chunks(chunks, output_root, base_filename, metadata, df_noise, b_save=True)

        if len(sound_idxs)>0:
            # Create Metadata DataFrame
            out_df = pd.DataFrame(metadata)
            
            # Filter based on analyzed indices
            out_df = out_df.iloc[sound_idxs]
            
            # Save results to file
            
            out_df.to_csv(output_file, index=False)
        else:
            # Empty csv
            with open(output_file, mode="w") as file:
                file.write("file,channel,start_ms,end_ms\n")
            print("no mosquito sound found.")
